In [4]:
# pip install pydub

In [7]:
import os
import random
from pydub import AudioSegment
import numpy as np

def generate_white_noise(duration_ms, amplitude=0.05):
    """
    Tạo nhiễu trắng với độ dài và biên độ cho trước.
    
    :param duration_ms: Độ dài của nhiễu (ms).
    :param amplitude: Biên độ của nhiễu (0.0 - 1.0, mặc định 0.05 để nhiễu nhỏ hơn).
    :return: AudioSegment của nhiễu trắng.
    """
    sample_rate = 44100  # Tần số mẫu mặc định
    num_samples = int(sample_rate * duration_ms / 1000)
    noise = np.random.uniform(-amplitude, amplitude, num_samples) * 32767  # 16-bit audio
    return AudioSegment(noise.tobytes(), frame_rate=sample_rate, sample_width=2, channels=1)

def add_noise_to_audio(original_audio_path, noise_audio_path=None, output_path=None, noise_reduction_factor=32, noise_type="white"):
    """
    Thêm nhiễu vào file audio gốc với các tùy chọn nhiễu khác nhau.
    
    :param original_audio_path: Đường dẫn đến file audio gốc.
    :param noise_audio_path: Đường dẫn đến file audio nhiễu (nếu dùng file).
    :param output_path: Đường dẫn lưu file audio kết quả.
    :param noise_reduction_factor: Hệ số giảm nhiễu (mặc định 32).
    :param noise_type: Loại nhiễu ('file' - từ file, 'white' - nhiễu trắng).
    """
    # Load audio gốc
    original_audio = AudioSegment.from_mp3(original_audio_path)
    duration_ms = len(original_audio)
    
    # Tạo hoặc load nhiễu tùy theo loại
    if noise_type == "file" and noise_audio_path:
        noise_audio = AudioSegment.from_wav(noise_audio_path)
        if len(noise_audio) < duration_ms:
            noise_audio = noise_audio * (duration_ms // len(noise_audio) + 1)

        noise_audio = noise_audio - 10  # Giảm thêm 20 dB
        noise_audio = noise_audio[:duration_ms]
    elif noise_type == "white":
        noise_audio = generate_white_noise(duration_ms, amplitude=0.05)  # Nhiễu trắng với biên độ thấp
        # Giảm thêm âm lượng của nhiễu trắng
        noise_audio = noise_audio - 30  # Giảm thêm 20 dB
    else:
        raise ValueError("Loại nhiễu không hợp lệ hoặc thiếu đường dẫn file nhiễu!")
    
    # Tính năng lượng của tín hiệu gốc và nhiễu
    original_rms = original_audio.rms
    noise_rms = noise_audio.rms
    
    # Điều chỉnh mức âm lượng của nhiễu
    if noise_rms > 0:
        target_rms = original_rms / noise_reduction_factor
        gain_needed = 20 * (target_rms / noise_rms)  # Tính gain theo dB
        noise_audio = noise_audio.apply_gain(gain_needed)
        print(f'Chỉnh lại âm lượng nhiễu ({noise_type}) xuống 1/{noise_reduction_factor} âm gốc')
    else:
        print(f'KHÔNG Chỉnh lại âm lượng (nhiễu có RMS = 0)')
    
    # Trộn tín hiệu gốc với nhiễu
    mixed_audio = original_audio.overlay(noise_audio)
    
    # Lưu file kết quả
    mixed_audio.export(output_path, format="mp3")

def process_audio_lists(original_audio_folder, noise_audio_folder, output_folder, noise_reduction_factor=32):
    """
    Thêm nhiều loại nhiễu vào tất cả file trong danh sách audio gốc.
    
    :param original_audio_folder: Thư mục chứa audio gốc.
    :param noise_audio_folder: Thư mục chứa audio nhiễu.
    :param output_folder: Thư mục để lưu file kết quả.
    :param noise_reduction_factor: Hệ số giảm nhiễu (mặc định 32).
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    original_files = [f for f in os.listdir(original_audio_folder) if f.endswith(".mp3")]
    # noise_files = [f for f in os.listdir(noise_audio_folder) if f.endswith(".wav")]
    
    for original_file in original_files:
        original_path = os.path.join(original_audio_folder, original_file)
        base_name = os.path.splitext(original_file)[0]
        
        # 1. Thêm nhiễu từ file
        # if noise_files:
        #     noise_file = random.choice(noise_files)
        #     noise_path = os.path.join(noise_audio_folder, noise_file)
        #     output_file = f"{base_name}.mp3"
        #     output_path = os.path.join(output_folder, output_file)
        #     add_noise_to_audio(original_path, noise_path, output_path, noise_reduction_factor, noise_type="file")
        #     print(f"Processed {original_file} with file noise {noise_file} -> {output_path}")
        
        # 2. Thêm nhiễu trắng
        output_file = f"{base_name}.mp3"
        output_path = os.path.join(output_folder, output_file)
        add_noise_to_audio(original_path, None, output_path, noise_reduction_factor, noise_type="white")
        print(f"Processed {original_file} with white noise -> {output_path}")

# Ví dụ sử dụng:
# process_audio_lists("path_to_original_audio", "path_to_noise_audio", "path_to_output", noise_reduction_factor=32)

In [8]:
process_audio_lists("./SampleAudioClean/data", "./SampleAudioClean/noise", "./add_noise_output_1", noise_reduction_factor=128)

Chỉnh lại âm lượng nhiễu (white) xuống 1/128 âm gốc
Processed FPTOpenSpeechData_Set001_V0.1_000001.mp3 with white noise -> ./add_noise_output_1/FPTOpenSpeechData_Set001_V0.1_000001.mp3
Chỉnh lại âm lượng nhiễu (white) xuống 1/128 âm gốc
Processed FPTOpenSpeechData_Set001_V0.1_000008.mp3 with white noise -> ./add_noise_output_1/FPTOpenSpeechData_Set001_V0.1_000008.mp3
Chỉnh lại âm lượng nhiễu (white) xuống 1/128 âm gốc
Processed FPTOpenSpeechData_Set001_V0.1_000004.mp3 with white noise -> ./add_noise_output_1/FPTOpenSpeechData_Set001_V0.1_000004.mp3
Chỉnh lại âm lượng nhiễu (white) xuống 1/128 âm gốc
Processed FPTOpenSpeechData_Set001_V0.1_000003.mp3 with white noise -> ./add_noise_output_1/FPTOpenSpeechData_Set001_V0.1_000003.mp3
Chỉnh lại âm lượng nhiễu (white) xuống 1/128 âm gốc
Processed FPTOpenSpeechData_Set001_V0.1_000006.mp3 with white noise -> ./add_noise_output_1/FPTOpenSpeechData_Set001_V0.1_000006.mp3
Chỉnh lại âm lượng nhiễu (white) xuống 1/128 âm gốc
Processed FPTOpenSpeech